<a href="https://colab.research.google.com/github/kartikgandhi/Cross-Project-Hard-Voting-Edited-Dataset/blob/main/Cross_Project_HV_train_gameextractor%2Btest_maven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from numpy import mean
from numpy import std

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [32]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [33]:
#file to initalize the weights from
weights = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-game_extractor 3.09 - game_extractor 3.09.csv")


In [34]:
weights['change'] = weights['change'].map({'YES': 1, 'NO': 0})

In [35]:
weights.shape

(2013, 8)

In [36]:
weights.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,13,0,39,328,1,89,81,1
1,3,0,46,60,2,0,15,1
2,1,0,73,32,2,0,10,1
3,3,0,12,139,1,79,34,1
4,4,1,32,245,1,79,58,1


In [37]:
X = weights.drop('change', axis=1)
y = weights['change']

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# split the full train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=1)

In [39]:
# get a list of base models
def get_models():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))
	return models

In [40]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# fit the model
		model.fit(X_train, y_train)
		# evaluate the model
		yhat = model.predict(X_val)
		acc = accuracy_score(y_val, yhat)
		# store the performance
		scores.append(acc)
		# report model performance
	return scores

In [41]:
# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train, X_val, y_train, y_val)
print(scores)

[0.7834586466165413, 0.7879699248120301, 0.7909774436090226]


In [42]:
data = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-maven 3.3.3-3.3.9 - maven 3.3.3-3.3.9.csv")

In [43]:
data['Change'] = data['Change'].map({'yes': 1, 'no': 0})

In [44]:
data.shape

(831, 8)

In [45]:
data.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,Change
0,22,10,26,237,1,73,13,0
1,2,0,18,40,0,40,5,0
2,4,0,24,71,0,27,11,0
3,2,2,16,15,0,0,3,0
4,3,0,14,7,0,0,1,0


In [46]:
X = data.drop('Change', axis=1)
y = data['Change']

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [48]:
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='hard', weights=scores)
# fit the ensemble on the training dataset
ensemble=ensemble.fit(X_train, y_train)
# make predictions on test set
yhat = ensemble.predict(X_test)

In [49]:
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('Weighted Avg Accuracy: %.3f' % (score*100))

Weighted Avg Accuracy: 77.844


In [50]:
print(yhat)

[1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0]


In [51]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,yhat)
print(cm)

[[115   2]
 [ 35  15]]


In [52]:
# predict probabilities
# from sklearn.metrics import roc_auc_score
# pred_prob = ensemble.predict_proba(X_test)
# auc_score = roc_auc_score(y_test, pred_prob[:,1])
# print("Area Under Curve=")
# print(auc_score)

In [53]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,yhat)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.4284991599549739


In [54]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, yhat)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.6414529914529914


In [55]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, yhat)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.5430209893473684


In [56]:
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.77      0.98      0.86       117
           1       0.88      0.30      0.45        50

    accuracy                           0.78       167
   macro avg       0.82      0.64      0.65       167
weighted avg       0.80      0.78      0.74       167

